<a href="https://colab.research.google.com/github/Wubetie/Wubetie/blob/main/ST_GCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
#!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
#!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
#!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-geometric
!pip install torch-geometric-temporal


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=998282fa8febbf9ac1208dcdac16c0354c95b2ced0ee5bc0f9f4595f333afcc6
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=495091 sha256=4a2d3a52baa8d18e5a7f02b6df54c216b005eab38e7a5bc4f0e7e94c829f355f


In [4]:
import math
import numpy as np  # to do maths operation
import pandas as pd
#import pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import ChebConv
import torch.optim as optim
import torch_geometric_temporal
import tensorflow as tf
import keras
#!pip install scipy
import scipy

In [5]:
# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Colab Notebooks/STGCN/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

#create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Colab Notebooks/STGCN/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
!ls /content/drive/MyDrive/Colab Notebooks/STGCN/models*.py

ls: cannot access '/content/drive/MyDrive/Colab': No such file or directory
ls: cannot access 'Notebooks/STGCN/models*.py': No such file or directory


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/STGCN')
#sys.path.append('https://github.com/MrGemy95/Tensorflow-Project-Template/blob/master')
sys.path.append('https://github.com/cmcuza/sst-gcn/blob/main/requirements')

In [16]:
import models
import utils
import requirements
import dataset

In [20]:
PeMSD7_V_228 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/STGCN/dataset/PeMSD7_V_228.csv')
PeMSD7_M_Station_Info=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/STGCN/dataset/PeMSD7_M_Station_Info.csv')

In [22]:
#PeMSD7_M_Station_Info.head
#PeMSD7_V_228.head()

,Data_Type,Region,ZONENAME,Zone_code,HHID,fid_2,lat,lng,year,rural,...,CSI,Geo_PCA1,Geo_PCA2,Geo_PCA3,PCA_Drinking_Water,PCA_Sanitation,PCA_Agr_pack,PCA_Mix_Irr_croping,PCA_non_agri_Bzns,FCSL
0,Train,Tigray,North Western,Tigray_Z1,1,44,14.0,38.0,2012,-0.370534,...,-0.498802,-0.71,-1.83,-0.10,1.20,1.20,0.09,-0.15,-0.97,-0.278500
1,Train,Tigray,North Western,Tigray_Z1,1,44,14.0,38.0,2014,-0.370534,...,-0.498802,-0.71,-1.85,-0.03,1.02,-1.48,0.96,0.22,-0.37,-0.278500
2,Train,Tigray,North Western,Tigray_Z1,1,44,14.0,38.0,2016,-0.370534,...,-0.352931,-0.70,-1.86,-0.02,1.17,-1.50,-0.12,0.06,0.06,-1.494031
3,Train,Tigray,North Western,Tigray_Z1,2,44,14.0,38.0,2012,-0.370534,...,-0.498802,-0.70,-1.78,-0.16,1.16,-1.55,0.09,-0.15,-0.97,-0.278500
4,Train,Tigray,North Western,Tigray_Z1,2,44,14.0,38.0,2014,-0.370534,...,-0.498802,-0.67,-1.88,-0.08,0.97,-0.20,1.93,-0.39,-0.10,-1.494031


In [ ]:
# @Time     : Jan. 02, 2019 22:17
# @Author   : Veritas YIN
# @FileName : main.py
# @Version  : 1.0
# @Project  : Orion
# @IDE      : PyCharm
# @Github   : https://github.com/VeritasYin/Project_Orion

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from os.path import join as pjoin

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

from utils.math_graph import *
from data_loader.data_utils import *
from models.trainer import model_train
from models.tester import model_test

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--n_route', type=int, default=228)
parser.add_argument('--n_his', type=int, default=12)
parser.add_argument('--n_pred', type=int, default=9)
parser.add_argument('--batch_size', type=int, default=50)
parser.add_argument('--epoch', type=int, default=50)
parser.add_argument('--save', type=int, default=10)
parser.add_argument('--ks', type=int, default=3)
parser.add_argument('--kt', type=int, default=3)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--opt', type=str, default='RMSProp')
parser.add_argument('--graph', type=str, default='default')
parser.add_argument('--inf_mode', type=str, default='merge')

args = parser.parse_args()
print(f'Training configs: {args}')

n, n_his, n_pred = args.n_route, args.n_his, args.n_pred
Ks, Kt = args.ks, args.kt
# blocks: settings of channel size in st_conv_blocks / bottleneck design
blocks = [[1, 32, 64], [64, 32, 128]]

# Load wighted adjacency matrix W
if args.graph == 'default':
    W = weight_matrix(pjoin('./dataset', f'PeMSD7_W_{n}.csv'))
else:
    # load customized graph weight matrix
    W = weight_matrix(pjoin('./dataset', args.graph))

# Calculate graph kernel
L = scaled_laplacian(W)
# Alternative approximation method: 1st approx - first_approx(W, n).
Lk = cheb_poly_approx(L, Ks, n)
tf.add_to_collection(name='graph_kernel', value=tf.cast(tf.constant(Lk), tf.float32))

# Data Preprocessing
data_file = f'PeMSD7_V_{n}.csv'
n_train, n_val, n_test = 34, 5, 5
PeMS = data_gen(pjoin('./dataset', data_file), (n_train, n_val, n_test), n, n_his + n_pred)
print(f'>> Loading dataset with Mean: {PeMS.mean:.2f}, STD: {PeMS.std:.2f}')

if __name__ == '__main__':
    model_train(PeMS, blocks, args)
    model_test(PeMS, PeMS.get_len('test'), n_his, n_pred, args.inf_mode)


AttributeError: ignored

# New Section